In [37]:
from GNBG import GNBG 
import numpy as np
from scipy.stats import norm
from scipy.optimize import fminbound
from copy import deepcopy
from scipy.optimize import OptimizeResult
problems = [GNBG(1), GNBG(2), GNBG(3), GNBG(4)]
function = [problems[i].fitness_of_ind for i in range(len(problems))]
K = len(function)

ID: 1
Optimal value:  -1081.9837994003399
ID: 2
Optimal value:  -703.1328146165181
ID: 3
Optimal value:  -357.5797495903721
ID: 4
Optimal value:  -382.6205211774271


In [38]:
N = 100 #pop_size
D = 30 #dim
sbxdi= 10
pmdi=10
pswap= 0.5
population = np.random.rand(2 * N, D)
skill_factor = np.array([i % K for i in range(2 * N)])
factorial_cost = np.full([2 * N, K], np.inf)
scalar_fitness = np.empty([2 * N])


In [39]:
skill_factor 

array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
       2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,
       0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
       2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,
       0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
       2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,
       0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
       2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,
       0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
       2, 3])

In [40]:
d = [8, 3, 5, 2, 6, 7, 1, 4]
np.argsort(np.argsort(d))+1

array([8, 3, 5, 2, 6, 7, 1, 4], dtype=int64)

In [41]:

for i in range(2*N):
    sf = skill_factor[i]
    factorial_cost[i, sf] = function[sf](population[i])

def calculate_scalar_fitness(factorial_cost):
    return 1 / np.min(np.argsort(np.argsort(factorial_cost, axis=0), axis=0) + 1, axis=1)
    ## np.argsort(np.argsort(d))+1 return the rank of individuals

scalar_fitness = calculate_scalar_fitness(factorial_cost)

In [42]:
sort_idx = np.argsort(scalar_fitness)[::-1]
population = population[sort_idx]
skill_factor = skill_factor[sort_idx]
scalar_fitness = scalar_fitness[sort_idx]

In [43]:
def get_subpops(population, skill_factor, N):
    K = len(set(skill_factor))
    subpops = []
    for k in range(K):
        idx = np.where(skill_factor == k)[0][:N//K]
        subpops.append(population[idx,:])
    return subpops

In [44]:
class Model:
    def __init__(self, mean, std, num_sample):
        self.mean        = mean
        self.std         = std
        self.num_sample  = num_sample

    def density(self, subpop):
        N, D = subpop.shape
        prob = np.ones([N])
        for d in range(D):
            prob *= norm.pdf(subpop[:, d], loc=self.mean[d], scale=self.std[d])
        return prob
  

In [45]:
def log_likelihood(rmp, prob_matrix, K):
  posterior_matrix = deepcopy(prob_matrix)
  value = 0
  for k in range(2):
    for j in range(2):
      if k == j:
        posterior_matrix[k][:, j] = posterior_matrix[k][:, j] * (1 - 0.5 * (K - 1) * rmp / float(K))
      else:
        posterior_matrix[k][:, j] = posterior_matrix[k][:, j] * 0.5 * (K - 1) * rmp / float(K)
    value = value + np.sum(-np.log(np.sum(posterior_matrix[k], axis=1)))
  return value


In [46]:
def learn_models(subpops):
  K = len(subpops)
  D = subpops[0].shape[1]
  models = []
  for k in range(K):
    subpop            = subpops[k]
    num_sample        = len(subpop)
    num_random_sample = int(np.floor(0.1 * num_sample))
    rand_pop          = np.random.rand(num_random_sample, D)
    mean              = np.mean(np.concatenate([subpop, rand_pop]), axis=0)
    std               = np.std(np.concatenate([subpop, rand_pop]), axis=0)
    models.append(Model(mean, std, num_sample))
  return models

def learn_rmp(subpops, D):
  K          = len(subpops)
  rmp_matrix = np.eye(K)
  models = learn_models(subpops)

  for k in range(K - 1):
    for j in range(k + 1, K):
      probmatrix = [np.ones([models[k].num_sample, 2]), 
                    np.ones([models[j].num_sample, 2])]
      probmatrix[0][:, 0] = models[k].density(subpops[k])
      probmatrix[0][:, 1] = models[j].density(subpops[k])
      probmatrix[1][:, 0] = models[k].density(subpops[j])
      probmatrix[1][:, 1] = models[j].density(subpops[j])

      rmp = fminbound(lambda rmp: log_likelihood(rmp, probmatrix, K), 0, 1)
      rmp += np.random.randn() * 0.01
      rmp = np.clip(rmp, 0, 1)
      rmp_matrix[k, j] = rmp
      rmp_matrix[j, k] = rmp

  return rmp_matrix

In [47]:
def sbx_crossover(p1, p2, sbxdi):
  D = p1.shape[0]
  cf = np.empty([D])
  u = np.random.rand(D)        

  cf[u <= 0.5] = np.power((2 * u[u <= 0.5]), (1 / (sbxdi + 1)))
  cf[u > 0.5] = np.power((2 * (1 - u[u > 0.5])), (-1 / (sbxdi + 1)))

  c1 = 0.5 * ((1 + cf) * p1 + (1 - cf) * p2)
  c2 = 0.5 * ((1 + cf) * p2 + (1 - cf) * p1)

  c1 = np.clip(c1, 0, 1)
  c2 = np.clip(c2, 0, 1)

  return c1, c2
def mutate(p, pmdi):
  mp = float(1. / p.shape[0])
  u = np.random.uniform(size=[p.shape[0]])
  r = np.random.uniform(size=[p.shape[0]])
  tmp = np.copy(p)
  for i in range(p.shape[0]):
    if r[i] < mp:
      if u[i] < 0.5:
        delta = (2*u[i]) ** (1/(1+pmdi)) - 1
        tmp[i] = p[i] + delta * p[i]
      else:
        delta = 1 - (2 * (1 - u[i])) ** (1/(1+pmdi))
        tmp[i] = p[i] + delta * (1 - p[i])
  tmp = np.clip(tmp, 0, 1)
  return tmp
def find_relative(population, skill_factor, sf, N):
  return population[np.random.choice(np.where(skill_factor[:N] == sf)[0])]
def variable_swap(p1, p2, probswap):
  D = p1.shape[0]
  swap_indicator = np.random.rand(D) <= probswap
  c1, c2 = p1.copy(), p2.copy()
  c1[np.where(swap_indicator)] = p2[np.where(swap_indicator)]
  c2[np.where(swap_indicator)] = p1[np.where(swap_indicator)]
  return c1, c2
def get_best_individual(population, factorial_cost, scalar_fitness, skill_factor, sf):
  # select individuals from task sf
  idx                = np.where(skill_factor == sf)[0]
  subpop             = population[idx]
  sub_factorial_cost = factorial_cost[idx]
  sub_scalar_fitness = scalar_fitness[idx]

  # select best individual
  idx = np.argmax(sub_scalar_fitness)
  x = subpop[idx]
  fun = sub_factorial_cost[idx, sf]
  return x, fun

def get_optimization_results(population, factorial_cost, scalar_fitness, skill_factor, message):
  K = len(set(skill_factor))
  N = len(population) // 2
  results = []
  for k in range(K):
    result         = OptimizeResult()
    x, fun         = get_best_individual(population, factorial_cost, scalar_fitness, skill_factor, k)
    result.x       = x
    result.fun     = fun
    result.message = message
    results.append(result)
  return results

In [48]:
from tqdm import tqdm
while(sum([problems[i].FE for i in range(len(problems))]) < 500000):
    print(sum([problems[i].FE for i in range(len(problems))]))
    # permute current population
    permutation_index = np.random.permutation(N)
    population[:N] = population[:N][permutation_index]
    skill_factor[:N] = skill_factor[:N][permutation_index]
    factorial_cost[:N] = factorial_cost[:N][permutation_index]
    factorial_cost[N:] = np.inf

    # learn rmp
    subpops    = get_subpops(population, skill_factor, N)
    rmp_matrix = learn_rmp(subpops, D)

    # select pair to crossover
    for i in range(0, N, 2):
      p1, p2 = population[i], population[i + 1]
      sf1, sf2 = skill_factor[i], skill_factor[i + 1]

      # crossover
      if sf1 == sf2:
        c1, c2 = sbx_crossover(p1, p2, sbxdi)
        c1 = mutate(c1, pmdi)
        c2 = mutate(c2, pmdi)
        c1, c2 = variable_swap(c1, c2, pswap)
        skill_factor[N + i] = sf1
        skill_factor[N + i + 1] = sf1
      elif sf1 != sf2 and np.random.rand() < rmp_matrix[sf1, sf2]:
        c1, c2 = sbx_crossover(p1, p2, sbxdi)
        c1 = mutate(c1, pmdi)
        c2 = mutate(c2, pmdi)
        # c1, c2 = variable_swap(c1, c2, pswap)
        if np.random.rand() < 0.5: skill_factor[N + i] = sf1
        else: skill_factor[N + i] = sf2
        if np.random.rand() < 0.5: skill_factor[N + i + 1] = sf1
        else: skill_factor[N + i + 1] = sf2
      else:
        p2  = find_relative(population, skill_factor, sf1, N)
        c1, c2 = sbx_crossover(p1, p2, sbxdi)
        c1 = mutate(c1, pmdi)
        c2 = mutate(c2, pmdi)
        c1, c2 = variable_swap(c1, c2, pswap)
        skill_factor[N + i] = sf1
        skill_factor[N + i + 1] = sf1

      population[N + i, :], population[N + i + 1, :] = c1[:], c2[:]

    # evaluate
    for i in range(N, 2 * N):
      sf = skill_factor[i]
      factorial_cost[i, sf] = function[sf](population[i])
    scalar_fitness = calculate_scalar_fitness(factorial_cost)

    # sort
    sort_index = np.argsort(scalar_fitness)[::-1]
    population = population[sort_index]
    skill_factor = skill_factor[sort_index]
    factorial_cost = factorial_cost[sort_index]

    best_fitness = np.min(factorial_cost, axis=0)
    c1 = population[np.where(skill_factor == 0)][0]
    c2 = population[np.where(skill_factor == 1)][0]
    scalar_fitness = scalar_fitness[sort_index]

    # optimization info
    message = {'algorithm': 'mfeaii', 'rmp':round(rmp_matrix[0, 1], 1)}
    results = get_optimization_results( population, factorial_cost, scalar_fitness, skill_factor, message)

  0%|          | 0/3000 [00:06<?, ?it/s]


200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18

[ message: algorithm: mfeaii
                rmp: 0.0
     fun: 70614.20109919771
       x: [ 2.574e-13  1.000e+00 ...  1.165e-13  1.000e+00],  message: algorithm: mfeaii
                rmp: 0.0
     fun: -701.3900981721936
       x: [ 5.511e-10  1.000e+00 ...  1.000e+00  1.225e-10],  message: algorithm: mfeaii
                rmp: 0.0
     fun: 40987274962.97319
       x: [ 4.198e-14  4.062e-13 ...  1.811e-13  1.000e+00],  message: algorithm: mfeaii
                rmp: 0.0
     fun: 99218.88998829968
       x: [ 3.591e-13  6.270e-12 ...  1.140e-13  1.000e+00]]
